# Post-impact temperature files & cooling setup
### Sarah Steele, 2023


## Utilities

In [1]:
## import important things
import os
import shutil

import numpy as np
import scipy.linalg
import scipy.integrate as integrate
from scipy.interpolate import interp1d
from scipy.interpolate import RectBivariateSpline 
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns

import gmsh

In [2]:
# load important constants and parameters
# constants|
g = 3.70;               # gravitational acceleration at surface (m/s)

# material properties for pre-impact thermal profiles
# [regolith,crust, mantle]
bgmats_fluery24 = {    
    'phys_groups': ['regolith','crust','mantle'],       # physical groups                                 
    'depth':np.array([2.,35.,455.])*1000,               # depth (m)
    'k': np.array([0.3,2.,4.]),                         # thermal conductivity (W/m/K) (35 km crust w/ k=2.5, 2 km insulating megaregolith with k=0.3)
    'Cp': np.array([1000.,1000.,1142.]),                # specific heat capacity (J/K/kg)
    'kappa':np.array([1.04E-6,1.04E-6,1.04E-6]),        # thermal diffusivity (m^2 s^-1) 
    'rho':np.array([0.81*2900.,2900.,3380.])            # density (kg/m^3) (megaregolith density following formulation in Sori (2018) with 19% porosity, e.g. Xie, Zhang, & Huang (2022))
}

# material properties (assuming prescribed crustal lithology)
mats = {"Cp": bgmats_fluery24['Cp'][1],      # heat capacity (J k^-1 C^-1) 
        "K0": 19.3E9,  # adiabatic bulk modulus at zero pressure (GPa) (Gault & Heitowit, 1963)
        "n": 5.5,        # derivative of the bulk modulus (Gault & Heitowit, 1963)
        "rho": bgmats_fluery24['rho'][1],    # uncompressed density (kg/m^3) 
        "Dq":11700.,        # simple-to-complex crater transition diameter (Susorney+, 2016)
        "Tliq":1650.,    # liquidus temperature (K) (0 GPa liquidus for S-corrected lherzolite composition in Fig. 6B of Namur+ (2016))
        "Tsol": 1148.+273., # solidus temperature (K) (solidus following eqn in Fig. 6 caption of Namur+ (2016) @ 0 GPa)
        # "gradT":    ,# thermal gradient (C km^-1) (Babeyko & Zharkov (2000))
        "flux":   25.  ,  # heat flux (very approximately fits temperature at 35 km and 50 km in Figure 2 of Fleury+ (2024))
        "Tbase": 1050.,   # T at the base of the lithosphere (K) (Breuer+, 2007)
        "Tsurf": 440.    # T at the surface (K) (Padovan+ 2014)        
        }

# impact defaults
imdefs = {"vi"  :   42.5,     # impact velocity (km/s) (Le Feuvre & Wieczorek, 2008) 
          "theta":  np.pi/4 # impact angle (radians)
        }



# bgmats_thick = {
#     'depth':np.array([50.,])*1000,      # depth (m) (crust: Goossens+, 2022
#     'k': np.array([3.,3.,4.]),          # thermal conductivity (W/m/K) (Breuer+, 2007)
#     'Cp': np.array([,,1212.]),          # specific heat capacity (J/K/kg) (mantle: Michel+ 2013)
#     'kappa':np.array([,,7E-7]),         # thermal diffusivity (m^2 s^-1) (mantle: Michel+ 2013)
#     'rho':np.array([2500.,2500.,3500.])           # density (kg/m^3) (crust: Goossens+, 2022; mantle: Michel+, 2013)
# }

# bgmats_thin = {
#     'depth':np.array([26.,])*1000,      # depth (m) (crust: Sori, 2018)
#     'k': np.array([3.,3.,4.]),          # thermal conductivity (W/m/K) (Breuer+, 2007)
#     'Cp': np.array([,,1212.]),          # specific heat capacity (J/K/kg) (mantle: Michel+ 2013)
#     'kappa':np.array([,,7E-7]),         # thermal diffusivity (m^2 s^-1) (mantle: Michel+ 2013)
#     'rho':np.array([,3500.])           # density (kg/m^3) (crust: ; mantle: Michel+, 2013)
# }

# important stem locations
bms1 = 'BaseMeshes/'
cfgtemplate1 = 'config_template.cfg'

In [3]:
def mod_bgmats_for_mesh(bgmats,max_depth):
    for i,di in enumerate(bgmats['depth']):
        if di > max_depth:
            phys_groups = bgmats['phys_groups'][:i+1]                          
            depths = np.copy(bgmats['depth'][:i+1])
            depths[-1] = max_depth
            depths = np.insert(depths,0,0.)
            return depths, phys_groups
        
    depths = np.insert(bgmats['depth'],0,0.)    
    return depths, bgmats['phys_groups']



## Pre-impact thermal profiles

In [4]:
## pre-impact thermal profile functions

# pre-impact thermal profile
def pitp(t):return

def pitp_fromfile(filepath,xv,zv):
    # load temperatures
    x0grid, z0grid, Teq = np.load(filepath)
    
    if x0grid[0] == xv and z0grid[:,0] == zv:
        return Teq
    else:
        # interpolate onto new x and z grids
        Teq_intrp = RectBivariateSpline(x0grid[0],z0grid[:,0],Teq)

        # evaluate over new coordinates
        return Teq_interp(xv,zv)
    
def pitp_auto(depth, Tsurf=mats["Tsurf"],bgmats=bgmats_fluery24,flux=mats['flux']):
    #T = np.zeros(depth.shape)
    
    zlist = np.arange(0,3000000.,0.5)
    Tlist = np.zeros(zlist.shape)
    
    grad = flux/bgmats['k']
    
    Tlist[zlist<=bgmats['depth'][0]] = Tsurf + zlist[zlist<=bgmats['depth'][0]]*grad[0]/1000
    
    Tthresh = bgmats['depth'][0]*grad[0]/1000+Tsurf
    for i in range(1,len(bgmats['depth'])):
        
        inds = np.logical_and(zlist<=bgmats['depth'][i], zlist>bgmats['depth'][i-1])
        d0 = bgmats['depth'][i-1]
        
        Tlist[inds] = Tthresh+ (zlist[inds]-d0)*grad[i]/1000
        
        Tthresh += (bgmats['depth'][i]-d0)*grad[i]/1000
        
    Tint = interp1d(zlist, Tlist)
    return Tint(depth)


In [1]:
depths=np.arange(0.,60000.,50.)

plt.figure(figsize=(3,6))
plt.plot(pitp_auto(depths),depths/1000.)
plt.grid(True)
plt.gca().yaxis.set_inverted(True)

NameError: name 'np' is not defined

## Shock heating function

In [6]:
def shockT(depth,dist,Dtc,pitp,Tsurf,plot=0,vi=imdefs["vi"],theta=imdefs["theta"],n=mats["n"],K0=mats["K0"],Cp=mats["Cp"],rhot=mats["rho"],Dq = mats["Dq"]):
    '''
    Calculate post-impact shock heating using equations from Abramov et al. (2013) 
    inputs
    --------------
    depth: 
    dist:
    Dtr: transient crater diameter (km)

    parameters
    --------------
    vi: impact velocity (km/s)
    theta: impact angle (degrees)
    K0: adiabatic bulk modulus
    Cp: heat capacity (J k^-1 C^-1)
    rhot: target density
    '''
    Dtc=Dtc*1000
    Dtr=Dtc*1.2
    depth = depth*1000
    dist = dist*1000

    # projectile radius
    Dim = ((Dtc/1.16)*(vi*1000)**(-0.44)*g**(0.22)*np.sin(theta)**(-1/3))**(1/0.78)
    Rp = Dim/2
    
    print('projectile radius: ', Rp)

    # make r array
    r = np.sqrt((depth-Rp)**2+dist**2)

    # specific uncompressed target volume
    V0 = 1/rhot

    # pressure decay exponent
    k = 0.625*np.log10(vi)+1.25

    # pressure at r = Rp
    A = rhot*(vi*1000)**2/4 *np.sin(theta)

    # peak pressure
    P = A*(r/Rp)**(-k)
    P[r<Rp] = 0

    # waste heat
    dEw = 0.5*(P*V0-(2*K0*V0)/n)*(1-(P*n/K0+1)**(-1/n))+(K0*V0/(n*(1-n)))*(1-(P*n/K0+1)**(1-(1/n)))
    
    # temperature
    Tshock = dEw/Cp
    Tshock[Tshock<0] = 0

    # remove excavated material
    excinds = (dist)**2*(Dtr+4*Rp)/(Dtc**2)-0.25*(Dtr) - Rp + depth >= 0
    t2 =np.where(excinds[1:,:]!=excinds[0:-1,:])
    rollinds = [x for _, x in sorted(zip(t2[1],t2[0]))] 
    
    Tsadj = np.copy(Tshock)
    for i in [x for x,_ in sorted(zip(t2[1],t2[0]))]:
        Tsadj[:,i] = np.roll(Tshock[:,i],-rollinds[i]-1)
        Tsadj[-rollinds[i]-1:,i] = 0

    # central uplift
    Df = 0.91*Dtr**(1.125)/Dq**(0.09)
    Rcp = 0.22*(Df)/2
    cudis = 0.06*(Df/1000)**(1.1)*1000

    regul = depth[(dist<=Rcp)*(depth<=1.25*(0.25*Dtr))]
    regulr = dist[(dist<=Rcp)*(depth<=1.25*(0.25*Dtr))]
    
    dtemp = np.zeros(np.shape(Tshock))
    normr = np.max((regulr-Rcp)**2)
    normd = np.max(1.25*(0.25*Dtr)-regul)
    dtemp[(dist<=Rcp)*(depth<=(1.25*(0.25*Dtr)))] = cudis*(regulr-Rcp)**2*((1.25*(0.25*Dtr))-regul)/(normr*normd)
    hhh=depth+dtemp
    Tul = pitp(depth+dtemp)

    # plotting :)
    if plot:
        fig = plt.figure(figsize=(12,8))
        ax = fig.add_subplot(111)
        plt.contourf(dist/1000,depth/1000,Tul + Tshock,[20,45,70,95,120,200,500],cmap='YlOrRd')
        plt.colorbar();
        plt.contour(dist/1000,depth/1000,(dist)**2*(Dtr+4*Rp)/(Dtc**2)-0.25*(Dtr) - Rp + depth,[0])
        ax.set_aspect('equal')
        ax.invert_yaxis()

        fig = plt.figure(figsize=(12,8))
        ax = fig.add_subplot(111)
        plt.contourf(dist/1000,depth/1000,Tul + Tsadj,np.linspace(0,200,40),cmap='YlOrRd')
        plt.colorbar();
        ax.set_aspect('equal')
        ax.invert_yaxis()
        
    # make temperature dictionary
    Tout = {};
    
    Tout['Tul'] = Tul 
    Tout['Tsh'] = Tshock 
    Tout['Tshadj'] = Tsadj 
    Tout['Ttot'] = Tul + Tsadj 
    
    # make post-convective profile    
    Tsol = 1350+depth*(1850-1350)/500000
    Tliq = 1500+depth*(1965-1500)/500000

    Tthresh = 0.6*Tsol + 0.4*Tliq 
    Tthresh = np.maximum(Tthresh,pitp(depth,Tsurf=Tsurf))
    
    Tout['Tthresh'] = Tthresh
    
    Tout['Tconvadj'] = np.minimum(Tul + Tsadj, Tthresh)
    
    return Tout

## Mesh-making

In [7]:
def make_mesh_rect2D(save_fp,depths,dxs,width,phys_groups,preview=False,mesh_type='tet'):
    """
    Creates a 2D stacked rectangular mesh using Gmsh based on the provided depths.

    Parameters:
    save_fp (str): File path to save the generated mesh.
    depths (list or numpy array): A list or array of depths for each layer of the mesh.
    dxs (list or numpy array): A list or array of characteristic element sizes for each layer.
    width (float): The width of the rectangular mesh.
    max_depth (float): The maximum depth of the mesh.
    phys_groups (list of str): Names of the physical groups for each layer. Should be one element shorter than depths.
    preview (bool): If True, display a preview of the mesh. Default is False.
    mesh_type (str): Type of mesh to generate ('tet' for triangular/tetrahedral, 'quad' for quadrahedral). Default is 'tet'.

    Returns:
    None: The function generates a .msh file containing the mesh and saves it to the specified file path.

    Example:
    depths = [0., -10., -20., -30.]
    dxs = [1., 1., 1., 1.]
    phys_groups = ["Regolith", "Crust", "Mantle"]
    make_mesh_rect2D("mesh.msh", depths, dxs, width=10, max_depth=-40)
    """
    gmsh.initialize()
    gmsh.model.add("stacked_rect_mesh")
    print(dxs)
    # add points for top and intermediate layers
    for i, depth in enumerate(depths):
        gmsh.model.geo.addPoint(0,-depth,0,dxs[i],2*i+1)
        gmsh.model.geo.addPoint(width,-depth,0,dxs[i],2*i+2)

    # # add points for the bottom
    # gmsh.model.geo.addPoint(0,-max_depth,0,dxs[i],2*i+3)
    # gmsh.model.geo.addPoint(width,-max_depth,0,dxs[i],2*i+4)

    # make points list
    points = np.arange(1,2*i+3)
    print(points)
    
    # make top line
    gmsh.model.geo.addLine(2,1,1)

    tot_lines = 1
    tot_loops = 1
    # create lines and surfaces
    for i in range(1,len(points)-2,2):
        p1, p2, p3, p4 = points[i-1:i+3]

        # make lines
        gmsh.model.geo.addLine(p2, p4, tot_lines+1)
        gmsh.model.geo.addLine(p4, p3, tot_lines+3)
        gmsh.model.geo.addLine(p3, p1, tot_lines+2)

        # make curve loop and plane surface
        print(p1,p2,p3,p4)
        gmsh.model.geo.addCurveLoop([tot_lines,tot_lines+1,tot_lines+3,tot_lines+2],tot_loops,reorient = True)
        gmsh.model.geo.addPlaneSurface([tot_loops],tot_loops)

        # increment counters
        tot_loops += 1
        tot_lines += 3

    # sychronize mesh
    gmsh.model.geo.synchronize()

    # to generate quadrangles instead of triangles, add
    # if mesh_type == 'quad':
        # gmsh.model.mesh.setRecombine(2, pl)    # add to generate

    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 2)
    
    # make physical groups
    for i,pg in enumerate(phys_groups):
        gmsh.model.addPhysicalGroup(2,[i+1], i+1, pg)
    
    lines = np.arange(1,tot_lines+1)
    # set boundary conditions
    gmsh.model.addPhysicalGroup(1,[lines[-1]], 1, "Bottom")
    gmsh.model.addPhysicalGroup(1,lines[2::3], 4, "Left (center)")
    gmsh.model.addPhysicalGroup(1,[1], 3, "Top")
    gmsh.model.addPhysicalGroup(1,lines[1::3], 2, "Right")
    
    # mesh the mesh!
    gmsh.model.mesh.generate(2)
    gmsh.write(save_fp)

    # launch the GUI to see the results:
    if preview:
        gmsh.fltk.run()

    gmsh.finalize()


def make_mesh_rect3D(save_fp,depths,dxs,width,phys_groups,preview=False,mesh_type='tet'):
    """
    Creates a 3D square stacked rectangular mesh using Gmsh based on the provided depths.

    Parameters:
    save_fp (str): File path to save the generated mesh.
    depths (list or numpy array): A list or array of depths for each layer of the mesh.
    dxs (list or numpy array): A list or array of characteristic element sizes for each layer.
    width (float): The width of the rectangular mesh.
    max_depth (float): The maximum depth of the mesh.
    phys_groups (list of str): Names of the physical groups for each layer. Should be one element shorter than depths.
    preview (bool): If True, display a preview of the mesh. Default is False.
    mesh_type (str): Type of mesh to generate ('tet' for triangular/tetrahedral, 'quad' for quadrahedral). Default is 'tet'.

    Returns:
    None: The function generates a .msh file containing the mesh and saves it to the specified file path.

    Example:
    depths = [0., -10., -20., -30.]
    dxs = [1., 1., 1., 1.]
    phys_groups = ["Regolith", "Crust", "Mantle"]
    make_mesh_rect2D("mesh.msh", depths, dxs, width=10, max_depth=-40)
    """
    gmsh.initialize()
    gmsh.model.add("stacked_rect_mesh")
    print(dxs)
    # add points for top and intermediate layers
    for i, depth in enumerate(depths):
        gmsh.model.geo.addPoint(0,-depth,0,dxs[i],2*i+1)
        gmsh.model.geo.addPoint(width,-depth,0,dxs[i],2*i+2)

    # # add points for the bottom
    # gmsh.model.geo.addPoint(0,-max_depth,0,dxs[i],2*i+3)
    # gmsh.model.geo.addPoint(width,-max_depth,0,dxs[i],2*i+4)

    # make points list
    points = np.arange(1,2*i+3)
    print(points)
    
    # make top line
    gmsh.model.geo.addLine(2,1,1)

    tot_lines = 1
    tot_loops = 1
    # create lines and surfaces
    for i in range(1,len(points)-2,2):
        p1, p2, p3, p4 = points[i-1:i+3]

        # make lines
        gmsh.model.geo.addLine(p2, p4, tot_lines+1)
        gmsh.model.geo.addLine(p4, p3, tot_lines+3)
        gmsh.model.geo.addLine(p3, p1, tot_lines+2)

        # make curve loop and plane surface
        print(p1,p2,p3,p4)
        gmsh.model.geo.addCurveLoop([tot_lines,tot_lines+1,tot_lines+3,tot_lines+2],tot_loops,reorient = True)
        gmsh.model.geo.addPlaneSurface([tot_loops],tot_loops)

        # increment counters
        tot_loops += 1
        tot_lines += 3

    # sychronize mesh
    gmsh.model.geo.synchronize()

    # to generate quadrangles instead of triangles, add
    # if mesh_type == 'quad':
        # gmsh.model.mesh.setRecombine(2, pl)    # add to generate

    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 2)
    
    # make physical groups
    for i,pg in enumerate(phys_groups):
        gmsh.model.addPhysicalGroup(2,[i+1], i+1, pg)
    
    lines = np.arange(1,tot_lines+1)
    # set boundary conditions
    gmsh.model.addPhysicalGroup(1,[lines[-1]], 1, "Bottom")
    gmsh.model.addPhysicalGroup(1,lines[2::3], 4, "Left (center)")
    gmsh.model.addPhysicalGroup(1,[1], 3, "Top")
    gmsh.model.addPhysicalGroup(1,lines[1::3], 2, "Right")
    
    # mesh the mesh!
    gmsh.model.mesh.generate(2)
    gmsh.write(save_fp)

    # launch the GUI to see the results:
    if preview:
        gmsh.fltk.run()

    gmsh.finalize()

## Basin class

In [11]:
## basin class
class Basin:
    def __init__(self,bname,Df,bgmats=bgmats_fluery24):
        self.name = bname           # basin name
        self.Df = Df  # final diameter
        self.bgmats = bgmats
        
        # get the rest of the important parameters
        self.getparams()
        
        return
        
    def doallT(self,xvals,zvals,pitp,savefp,mesh_max_depth,mesh_width,mesh_dxs,mesh_preview=False,mesh_type='tet',bmstem=bms1,cfgtemp=cfgtemplate1,**shockkw):
        
        # make save dir if necessary
        if not os.path.isdir(savefp):
            os.mkdir(savefp)
        
        # set surface temperature
        self.Tsurf = pitp(0)
            
        # make temperature field
        print('Making post-impact T field')
        Tout = self.makeTfield(pitp,xvals,zvals,**shockkw)
        Tmat = Tout['Tconvadj']
        
        # save x, z, and T files
        xfp,zfp,Tfp = self.makexzT(savefp,xvals*1000,-zvals*1000,Tmat)
        
        # # choose appropriate mesh file
        # meshlims = np.array([[5,10,50,100,250,500],[10,50,100,250,500,800]])
        # meshinds = meshlims[:,(meshlims[0,:]<=self.Dtc)*(meshlims[1,:]>self.Dtc)]

        # basemeshfp = bmstem + str(meshinds[0,0]) + '_' + str(meshinds[1,0]) + 'km_rcm.inp'

        # print('Using mesh file ' + str(meshinds[0,0]) + '_' + str(meshinds[1,0]) + 'km_rcm.inp')
        
        # # modify and save new mesh 
        # newmeshfp = savefp + str(meshinds[0,0]) + '_' + str(meshinds[1,0]) + 'km_rcm.inp'
        # newmeshdf,xright,zbottom = self.makemesh(basemeshfp,savefp)
        #(save_fp,depths,dxs,width,max_depth,phys_groups,preview=False,mesh_type='tet')
        
        depths, phys_groups = mod_bgmats_for_mesh(self.bgmats,mesh_max_depth)
        
        make_mesh_rect2D(os.path.join(savefp,'mesh.msh'),depths,
                         mesh_dxs,mesh_width,phys_groups,
                         preview=mesh_preview,mesh_type=mesh_type)

        # make config file
        self.makeconfig(cfgtemp,savefp,xfp,zfp,Tfp,os.path.join(savefp,'mesh.msh'),mesh_width,-mesh_max_depth)
        
        # make output file
        outputfp = savefp + 'output';
        if not os.path.isdir(outputfp):
            os.mkdir(outputfp)
        
        return 
    
    
    
    def getparams(self):
        '''
        Calculate other important crater parameters.
        
        Dtr: transient crater diameter
        cudis: central uplift displacement
        Rim: estimated impactor radius
        '''
        
        self.Dtc = ((1/0.91)*mats["Dq"]**(0.09)*(self.Df*1000))**(1/1.125)/1.2/1000   # Abramov & Kring (2005)
        self.cudis = 0.06*(self.Df)**(1.1)                         # Grieve et al. (1981)
        self.uplift = 1
        
        return
        
    def makeTfield(self,pitp,xvals,zvals,**kwargs):
        '''
        Make np array of initial temperature profile.
        
        pitp: pre-impact temperature profile (C)
        rdist: radial extent of domain (km)
        zdist: depth extent of domain (km)
        res: resolution of domain (km)
        '''
        # set surface temperature
        self.Tsurf = pitp(0)
        
        # initialize base position arrays
        dist,depth = np.meshgrid(xvals,zvals)
        
        # get shock temperature profile
        Tmat = shockT(depth,dist,self.Dtc,pitp,self.Tsurf,**kwargs)
        
        self.Tibounds = [np.max(dist),np.max(depth)]

        
        print('Made temperature field')
        return Tmat
   

    def makexzT(self,filepath,xvals,zvals,Tmat,app=''):
        '''
        Make x, z, and T files. You should have already run makeTfield over a domain
        as large or larger than the x-z space used here.
        '''
        xfp = filepath + r'xc' + app + '.txt'
        zfp = filepath + r'zc' + app + '.txt'
        Tfp = filepath + 'T_smooth' + app + '.txt'
        Teqfp = filepath + 'T_eq_smooth' + app + '.txt'

        # write x and z files
        np.savetxt(xfp,xvals.reshape(-1,1),fmt='%6.5f')
        np.savetxt(zfp,zvals.reshape(-1,1)[::-1],fmt='%6.5f')
        print('Wrote x to ' + xfp)
        print('Wrote z to ' + zfp)
        
        # write T file
        np.savetxt(Tfp,Tmat[::-1],fmt='%.2f')
        print('Wrote T to ' + Tfp)
        
        # write equilibrium T file
        T_eqp = np.tile(Tmat[:,-1],(len(xvals),1)).T
        np.savetxt(Teqfp,T_eqp[::-1])
        
        return xfp, zfp, Tfp
    
    
    def makemesh(self,basemeshfp,savefp):
        '''
        Load mesh from file and:
        1. Add a melt sheet or ejecta blanket above the crater if desired
        2. Save mesh grid locations to mesh_params.txt file
        '''
        # load in mesh
        dfbase = pd.read_csv(basemeshfp, sep=" ",names=[1,2,3,4,5,6,7,8,9,10,11], skiprows=1, engine='python');
        
        sec1 = dfbase[dfbase[3]!='quad'].astype('float')
        
        shutil.copy2(basemeshfp,savefp + 'mesh.inp')
        shutil.copy2(basemeshfp[:-4]+'.svg',savefp + 'mesh.svg')
        print(basemeshfp[:-4]+'.svg')
            
        # find unique indices 
        xun = np.sort(sec1[3].unique())
        zun = np.sort(sec1[4].unique())[::-1]
        
        xunm, zunm = np.meshgrid(xun,zun)
        meshparams = np.concatenate((xunm.flatten().reshape(-1,1),zunm.flatten().reshape(-1,1)),axis=1)
        np.savetxt(savefp+'mesh_params.txt', meshparams)
            
        # get locations of right and bottom boundaries
        xright = sec1[3].max()
        zbottom = dfbase[4].min()
        
        return dfbase, xright, zbottom
    
    
    def makeconfig(self,templatefp,savefp,xfp,zfp,Tfp,meshfp,xright,zbottom):
        # make copy of config template
        cfgsave = savefp + 'config.cfg'
        shutil.copy(templatefp,cfgsave) 
        
        cfg = open(cfgsave, "r")

        list_of_lines = cfg.readlines()
        
        # set file paths
        list_of_lines[3] = 'mesh_filename = "' + meshfp +'";\n';
        list_of_lines[4] = 'output_folder = "' + savefp + 'output\"' + ';\n';
        list_of_lines[5] = 'x_file        = "' + xfp + '";\n';
        list_of_lines[6] = 'z_file        = "' + zfp + '";\n';
        list_of_lines[7] = 'temp_file     = "' + Tfp + '";\n';
        list_of_lines[8] = 'eq_temp_file     = "' + savefp + 'T_eq_smooth.txt' + '";\n';
        
        list_of_lines[14] = f'    k               = [' + ', '.join(['%.1f']*len(self.bgmats["k"]))% tuple(self.bgmats["k"])+'];        // double, thermal conductivity W/m/K\n'; 
        list_of_lines[15] = f'    cp              = [' + ', '.join(['%.1f']*len(self.bgmats["Cp"]))% tuple(self.bgmats["Cp"])+'];  // double, specific heat capacity J/K/kg\n'; 
        list_of_lines[16] = f'    rho             = [' + ', '.join(['%.1f']*len(self.bgmats["rho"]))% tuple(self.bgmats["rho"])+'];   // double, density, kg/m^3\n'; 
        # set surface temperature
        list_of_lines[17] = f'    T_surf          = {self.Tsurf:1.1f};             // double, surface temperature\n'

        # set material ids (gmsh cannot have 0 for material ids, but mars_heat treats them as indices)
        list_of_lines[29] = '    material_id                 = [' + ', '.join(['%d']*len(self.bgmats["rho"]))% tuple(range(1,len(self.bgmats["rho"])+1))+']; // regolith, crust, lithosphere, mantle\n' 
        
        # set mesh boundaries
        list_of_lines[73] = '    x_right  = %.1f;  // double\n'%xright;
        list_of_lines[74] = '    z_bottom = %.1f; // double\n'%zbottom;
        
        # save new config file
        cfg = open(cfgsave, "w")

        cfg.writelines(list_of_lines)
        cfg.close()
        
        return
        

## Workspace

In [12]:
xv100 = np.arange(0,100.1,0.25)
zv100 = np.arange(0,100.1,0.25)

n = 77
save_fp = r'/home/mike/Sarah/Mercury/MercuryBasins/77km/'
d100km = Basin('%d km diameter'%n,n)
dfbase=d100km.doallT(xv100,zv100,pitp_auto,save_fp,40000.,40000.,[250.,250.,500.,1000.],mesh_preview=False)



Making post-impact T field
projectile radius:  1449.0803118913866
Made temperature field
Wrote x to /home/mike/Sarah/Mercury/MercuryBasins/77km/xc.txt
Wrote z to /home/mike/Sarah/Mercury/MercuryBasins/77km/zc.txt
Wrote T to /home/mike/Sarah/Mercury/MercuryBasins/77km/T_smooth.txt
[250.0, 250.0, 500.0, 1000.0]
[1 2 3 4 5 6 7 8]
1 2 3 4
3 4 5 6
5 6 7 8
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 40%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 5 (Line)
Info    : [ 60%] Meshing curve 6 (Line)
Info    : [ 70%] Meshing curve 7 (Line)
Info    : [ 80%] Meshing curve 8 (Line)
Info    : [ 90%] Meshing curve 9 (Line)
Info    : [100%] Meshing curve 10 (Line)
Info    : Done meshing 1D (Wall 0.00345694s, CPU 0.003761s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [  0%] Simple recombination completed (Wall 0.00404254s, CPU 0.0040

In [18]:
bgmats_fluery24['depth']

array([  2000.,  35000., 455000.])